# Bicycle Accidents x Geo Data x Strava Exposure Merge


#### Import some standard libraries and helper scripts:

In [1]:
import sys
from pathlib import Path

# Ensure project root is importable when running from notebooks/
_project_root = Path.cwd().resolve()
if not (_project_root / "src").exists() and (_project_root.parent / "src").exists():
    _project_root = _project_root.parent
sys.path.insert(0, str(_project_root))

import datetime as dt
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from src.preprocess_data import preprocess_accident_data



In [2]:
# Kernel cleanup: drop old variables from pre-refactor runs
# (Equivalent to a kernel restart for the pipeline variables.)
import gc
import sys

# Ensure the removed module isn't lingering in memory
sys.modules.pop("src.merge_datasets", None)

# Drop previously computed objects/dataframes so we don't keep stale types around
for _name in [
    "seg",
    "clustering",
    "accidents_raw",
    "accidents_bike_berlin",
    "segment_geo_gdf",
    "segment_static",
    "strava_berlin_data",
    "summary_df",
    "final_exposure_ym",
    "accidents_agg_ym_rich",
    "merged_accidents_strava_ym",
    "core_panel",
    "nodes_raw",
    "node_points",
    "crossings_gdf",
    "crossing_ids",
    "segment_node_map",
    "node_exposure_ym",
    "node_panel_ym",
    "acc_node",
    "acc_node_ym",
    "joined_nearest_unique",
    "stats",
    "merge_keys",
    "min_year",
    "max_year",
    "out_dir",
    "out_path",
]:
    globals().pop(_name, None)

gc.collect()
print("kernel_cleanup_done")


kernel_cleanup_done


## Bicycle data for Berlin

In [3]:
accidents_bike_berlin = preprocess_accident_data(save_to_parquet=False)
accidents_bike_berlin.head()

Loaded 9 files -> combined shape: (2098019, 32)
Dropped irrelevant columns -> shape: (2098019, 24)
Filtered to bicycle accidents -> shape: (626844, 24)
Filtered to bicycle accidents in Berlin -> shape: (33181, 19)


,year,month,hour,weekday,injury_severity,accident_kind,accident_type,car_involved,pedestrian_involved,motorcycle_involved,other_vehicle_involved,road_condition,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,source_file,light_condition,goods_vehicle_involved
0,2018,1,15,4,3,6,7,0,1,0,0,1,798261.3849,5.835047e+06,13.403228,52.583472,Unfallorte2018_LinRef.csv,1,0
1,2018,1,11,5,3,5,2,1,0,0,0,1,800551.7210,5.829808e+06,13.432186,52.535255,Unfallorte2018_LinRef.csv,0,0
2,2018,1,8,2,3,5,2,1,0,0,0,0,803320.7292,5.827627e+06,13.470897,52.514173,Unfallorte2018_LinRef.csv,0,0
3,2018,1,19,4,3,5,7,1,0,0,0,1,798174.6913,5.826940e+06,13.394673,52.510848,Unfallorte2018_LinRef.csv,2,0
4,2018,1,18,4,2,5,3,1,0,0,0,1,806109.6297,5.821644e+06,13.506372,52.458993,Unfallorte2018_LinRef.csv,2,0


In [4]:
from src.segments import load_segment_geometry
seg = load_segment_geometry(canonical_crs="EPSG:32633")
CANONICAL_CRS = seg.canonical_crs

segment_geo_gdf = seg.segments_gdf
segment_static = seg.segment_static

segment_geo_gdf.head()


,geometry,counter_name,latitude,longitude
0,"LINESTRING (388283.894 5816533.578, 388349.119...",streetsegment_0,52.486743,13.355350
1,"LINESTRING (389240.438 5813521.134, 389260.513...",streetsegment_1,52.461885,13.369878
2,"LINESTRING (388562.846 5831195.503, 388578.563...",streetsegment_2,52.619820,13.354749
3,"LINESTRING (388683.345 5831306.663, 388717.453...",streetsegment_3,52.620476,13.357354
4,"LINESTRING (386530.508 5820675.884, 386544.692...",streetsegment_4,52.524039,13.328604


## Spatial Join: Accidents with Strava data (code from Luise and Eric) + edited by Tobi to achieve canonical geometry data


### Attempt 2: Use sjoin_nearest to assign exactly one (the nearest) segment to each accident
Challenges:
* need to find the right maximum distance so accidents that are not on a segment are not assigned to one.
* assigns two segments if their distance is equal

In [5]:
from src.accidents import assign_accidents_to_nearest_segment
# Assign each accident to exactly one nearest segment (within max_distance)
joined_nearest_unique = assign_accidents_to_nearest_segment(
    accidents_bike_berlin,
    segment_geo_gdf,
    canonical_crs=CANONICAL_CRS,
    max_distance_m=10,
)

print(f"Total accidents: {len(accidents_bike_berlin)}")
print(f"Total bike network Strava segments: {len(segment_geo_gdf)}")
print(f"Unique Strava segments in matched dataset: {joined_nearest_unique['counter_name'].nunique()}")
print(f"Accidents assigned to segments: {len(joined_nearest_unique)}")
print(f"Ratio of assigned accidents: {len(joined_nearest_unique) / len(accidents_bike_berlin):.2%}")

joined_nearest_unique.head()


Total accidents: 33181
Total bike network Strava segments: 4958
Unique Strava segments in matched dataset: 3570
Accidents assigned to segments: 21666
Ratio of assigned accidents: 65.30%


,year,month,hour,weekday,injury_severity,accident_kind,accident_type,car_involved,pedestrian_involved,motorcycle_involved,...,source_file,light_condition,goods_vehicle_involved,geometry,acc_id,index_right,counter_name,latitude,longitude,dist
29872,2021,4,9,4,2,1,5,1,0,0,...,Unfallorte_2021_LinRef.csv,0,0,POINT (397322.52 5813776.685),29872,4661.0,streetsegment_4661,52.461939,13.492277,0.000047
25448,2024,3,9,1,3,5,3,1,0,0,...,Unfallorte2024_LinRef.csv,0,0,POINT (377489.976 5821932.897),25448,4436.0,streetsegment_4436,52.533763,13.194189,0.000098
9074,2019,9,6,4,3,5,2,0,0,0,...,Unfallorte2019_LinRef.csv,1,0,POINT (405293.008 5812309.772),9074,2567.0,streetsegment_2567,52.452859,13.606787,0.000193
6943,2019,5,16,5,3,5,2,1,0,0,...,Unfallorte2019_LinRef.csv,0,0,POINT (385705.621 5826533.591),6943,1210.0,streetsegment_1210,52.577151,13.310916,0.000222
7115,2019,5,18,3,1,0,1,0,0,0,...,Unfallorte2019_LinRef.csv,0,0,POINT (410373.006 5803066.518),7115,4637.0,streetsegment_4637,52.371061,13.677376,0.000236


## Strava data (bicycle network traffic, other features - daily)

1. We need to aggregate this df to the same granularity as in Accidents data (segment, year, month, weekday) to join. 
2. We can not join only by geo data, as Accidents don't have date column, but Strava contains daily info (eg specific traffic volume or weather on specific day)
3. We can not just calculate mean of all columns in Strava data by year-month-etc... as we also have categorial features (for example `infrastructure_bicyclelane_type`) and some features are constant over time for segment (eg `infrastructure_max_speed` in dataset is constant for segment for all dates)

In [6]:
from src.strava_exposure import load_strava_berlin_data
strava_berlin_data = load_strava_berlin_data()
strava_berlin_data.columns.tolist()

['counter_name',
 'date',
 'count',
 'year',
 'latitude',
 'longitude',
 'geometry',
 'socioeconomic_gender_distribution',
 'infrastructure_bicyclelane_type',
 'infrastructure_type_of_street',
 'infrastructure_number_of_street_lanes',
 'infrastructure_street_smoothness',
 'infrastructure_street_surface',
 'infrastructure_max_speed',
 'infrastructure_cyclability',
 'weather_temp_avg',
 'weather_temp_min',
 'weather_temp_max',
 'weather_precipitation',
 'weather_snowfall',
 'weather_wind_speed_avg',
 'weather_wind_speed_gust',
 'weather_pressure',
 'weather_sunshine_duration',
 'strava_total_trip_count',
 'strava_ride_count',
 'day_of_week',
 'month']

### Which data types we have as features?

Results:

1. Mostly we have numerical features, but also categorical ones like `'infrastructure_bicyclelane_type'` - we will check if we need to aggregate them somehow or they are contstant over time.
2. Analysis shows:
    - **Numeric columns (111)**: Traffic counts, speeds, socioeconomic indicators, weather data
    - **Categorical columns**: Infrastructure types, activity types, street properties
    - **Boolean columns (8)**: Holiday flags, weekend indicators, data quality flags
3. **Key finding**: All connectivity and infrastructure columns are constant per segment, so they only need to be taken once per segment. Socioeconomic, motorized, strava, and weather columns vary over time and require aggregation by year-month-weekday.

In [7]:
df = strava_berlin_data
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=["number"]).columns.tolist()
bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()

print("Numeric:", len(numeric_cols))
print(numeric_cols)
print("\nCategorical:", len(categorical_cols))
print(categorical_cols)
print("\nBool:", len(bool_cols))
print(bool_cols)


Numeric: 15
['count', 'latitude', 'longitude', 'socioeconomic_gender_distribution', 'weather_temp_avg', 'weather_temp_min', 'weather_temp_max', 'weather_precipitation', 'weather_snowfall', 'weather_wind_speed_avg', 'weather_wind_speed_gust', 'weather_pressure', 'weather_sunshine_duration', 'strava_total_trip_count', 'strava_ride_count']

Categorical: 13
['counter_name', 'date', 'year', 'geometry', 'infrastructure_bicyclelane_type', 'infrastructure_type_of_street', 'infrastructure_number_of_street_lanes', 'infrastructure_street_smoothness', 'infrastructure_street_surface', 'infrastructure_max_speed', 'infrastructure_cyclability', 'day_of_week', 'month']

Bool: 0
[]


### Check which features we have contstant for one segment over time, so we don't need to aggregate them futher

In [8]:
from src.strava_exposure import column_stability_summary

summary_df = column_stability_summary(strava_berlin_data, group_col="counter_name")
summary_df.head(28)


,column,segments_total,segments_varying,max_unique_within_any_segment
0,latitude,4958,0,1
1,infrastructure_bicyclelane_type,4958,0,1
2,geometry,4958,0,1
3,longitude,4958,0,1
4,infrastructure_street_surface,4958,0,1
5,infrastructure_street_smoothness,4958,0,1
6,infrastructure_number_of_street_lanes,4958,0,1
7,infrastructure_type_of_street,4958,0,1
8,infrastructure_max_speed,4958,0,1
9,infrastructure_cyclability,4958,0,1


In [9]:
# tag dtypes
col_dtype = strava_berlin_data.dtypes
summary_df["dtype_bucket"] = summary_df["column"].map(
    lambda c: "bool" if col_dtype[c].name == "bool"
    else "numeric" if np.issubdtype(col_dtype[c], np.number)
    else "categorical"
)

# overall constant/varying summary
overall_stats = {
    "total_columns": len(summary_df),
    "constant_columns": int((summary_df["segments_varying"] == 0).sum()),
    "varying_columns": int((summary_df["segments_varying"] > 0).sum()),
}
overall_stats["percent_constant"] = round(
    overall_stats["constant_columns"]
    / max(overall_stats["total_columns"], 1)
    * 100,
    1,
 )

print("Overall column stability:")
for key, value in overall_stats.items():
    print(f"  {key}: {value}")

# dtype-level statistics
dtype_counts = (
    summary_df
    .groupby(["dtype_bucket"])
    .agg(
        total_cols=("column", "count"),
        constant_cols=("segments_varying", lambda s: (s == 0).sum()),
        varying_cols=("segments_varying", lambda s: (s > 0).sum()),
    )
)

dtype_counts["percent_constant"] = (
    dtype_counts["constant_cols"] / dtype_counts["total_cols"] * 100
).round(1)

display(dtype_counts.sort_values("percent_constant", ascending=False))

Overall column stability:
  total_columns: 27
  constant_columns: 10
  varying_columns: 17
  percent_constant: 37.0


,total_cols,constant_cols,varying_cols,percent_constant
dtype_bucket,,,,
categorical,12,8,4,66.7
numeric,15,2,13,13.3


### Result: All connectivity and infrastructure columns are constant per segment. Socioeconomic, Motorized and weather columns vary, so we need to aggregate them.

- Connectivity (7/7 constant, 2 bool, 5 numeric): treat as static attributes per segment; just carry a single value (e.g., first).
- Infrastructure (58/58 constant, 1 bool, 10 categorical, 47 numeric): fully static; keep one value per segment, no temporal aggregation needed.
**- Other (14 cols, 5 constant/9 varying; 5 bool/6 cat/3 num): mixed bag—decide column by column; reassign misfiled cols if any.**
- Motorized (12/12 varying, all numeric): fully time-varying; aggregate over your time buckets (sum for counts, mean for speeds).
- Socioeconomic (17/17 varying, numeric): varies across time in the data; aggregate over your time buckets (sum for counts, mean for speeds).
**- Strava (19/19 varying; 1 categorical, 18 numeric): counts/speeds should be summed/averaged per time bucket; handle the single categorical (strava_activity_type) via ????**
- Weather (9/9 varying, numeric): time-varying; aggregate with mean (or min/max if useful).



## Aggregation of Berlin Strava data 
1. Aggregation keys: counter_name (segment), year, month, weekday (to align with accidents).
2. Constant features stay as-is (no aggregation) since they don’t vary over time.

### This code execution can take a while, on Liaisan's pc ~13 minutes.

# Segment level risk

In [10]:
from src.strava_exposure import build_exposure_panel_segment_year_month
# --- 1. Build exposure panel from Strava/sensor data (segment x year x month) ---
final_exposure_ym = build_exposure_panel_segment_year_month(
    strava_berlin_data,
    segment_static=segment_static,
    summary_df=summary_df,
)

print("Exposure panel (segment–year–month) shape:", final_exposure_ym.shape)
final_exposure_ym.head()


KeyError: "Strava dataframe missing required columns: ['street_name', 'is_shortterm']"

## Aggregate accidents

In [ ]:
# --- 2. Build rich accident panel from Unfallatlas+segments (segment x year x month) ---
min_year = int(final_exposure_ym["year"].min())
max_year = int(final_exposure_ym["year"].max())

accidents_agg_ym_rich = aggregate_accidents_segment_year_month_rich(
    joined_nearest_unique,
    column_map=accident_columns_en,
    exposure_year_min=min_year,
    exposure_year_max=max_year,
)

print(
    "Rich accident aggregate (segment–year–month) shape:",
    accidents_agg_ym_rich.shape,
)
accidents_agg_ym_rich.head()


Rich accident aggregate (segment–year–month) shape: (14085, 64)


,counter_name,year,month,total_accidents,acc_involved_bicycle_count,acc_involved_passenger_car_count,acc_involved_pedestrian_count,acc_involved_motorcycle_count,acc_involved_goods_vehicle_count,acc_involved_other_vehicle_count,...,acc_accident_type_share_4,acc_accident_type_share_5,acc_accident_type_share_6,acc_accident_type_share_7,acc_light_condition_count_0.0,acc_light_condition_count_1.0,acc_light_condition_count_2.0,acc_light_condition_share_0.0,acc_light_condition_share_1.0,acc_light_condition_share_2.0
0,streetsegment_0,2019,1,1,1,1,0,0,0.0,0,...,0.0,0.0,0.0,1.0,1,0,0,1.0,0.0,0.0
1,streetsegment_0,2019,4,1,1,1,0,0,0.0,0,...,0.0,0.0,0.0,1.0,1,0,0,1.0,0.0,0.0
2,streetsegment_0,2019,7,1,1,0,0,1,0.0,0,...,0.0,0.0,0.0,0.0,1,0,0,1.0,0.0,0.0
3,streetsegment_0,2019,9,1,1,1,0,0,0.0,0,...,0.0,0.0,0.0,1.0,1,0,0,1.0,0.0,0.0
4,streetsegment_0,2020,5,1,1,1,0,0,0.0,0,...,0.0,1.0,0.0,0.0,0,0,1,0.0,0.0,1.0


## Merge datasets

In [ ]:
# --- 3. Merge exposure and accident panels into a risk panel ---

merge_keys = ["counter_name", "year", "month"]

merged_accidents_strava_ym = merge_exposure_and_accidents(
    final_exposure_ym,
    accidents_agg_ym_rich,
    merge_keys=merge_keys,
 )

print(
    "Merged risk panel (segment–year–month) shape:",
    merged_accidents_strava_ym.shape,
 )

merged_accidents_strava_ym.head()

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_strava_panel.parquet"

gpd.GeoDataFrame(
    merged_accidents_strava_ym,
    geometry="geometry",
    crs=segment_geo_gdf.crs,
 ).to_parquet(
    out_path,
    index=False,
 )

Merged risk panel (segment–year–month) shape: (297480, 193)


ImportError: Missing optional dependency 'pyarrow.parquet'. pyarrow is required for Parquet support.  "
        "Use pip or conda to install pyarrow.parquet.

## Sanity check of the merge

In [ ]:
stats = sanity_check_merge(
    merged_accidents_strava_ym=merged_accidents_strava_ym,
    accidents_agg_ym_rich=accidents_agg_ym_rich,
    final_exposure_ym=final_exposure_ym,
)

for k, v in stats.items():
    print(f"{k}: {v}")


# Playground, to create and test smaller dataset version. The smaller ones are created based on the completly merged version.

In [ ]:
core_panel = build_core_risk_panel(merged_accidents_strava_ym)
print("Core panel shape:", core_panel.shape)
core_panel.head(20)

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_strava_risk_core_panel.parquet"

gpd.GeoDataFrame(
    core_panel,
    geometry="geometry",
    crs=segment_geo_gdf.crs,
 ).to_parquet(
    out_path,
    index=False,
 )

In [ ]:
# find unique numbers of accidents in core_panel
unique_accident_counts = core_panel["total_accidents"].nunique()
print(f"Unique accident counts in core panel: {unique_accident_counts}")

# display those unique counts
print("Unique accident counts:", core_panel["total_accidents"].unique())




In [ ]:
# sum up the accidents per segment to verfiy total accidents per segment
accidents_per_segment = (
    core_panel
    .groupby("counter_name", as_index=False)["total_accidents"]
    .sum()
    .rename(columns={"total_accidents": "total_accidents_segment"})
)

# find segments with highest total accidents
top_segments = accidents_per_segment.sort_values("total_accidents_segment", ascending=False).head(10)
print("Top 10 segments by total accidents:")
display(top_segments)

# Crossing (junction) risk

### Build nodes (junction candidates) from segment endpoints

In [ ]:
nodes_raw = build_nodes_from_segment_endpoints(
    segment_geo_gdf,
    counter_col="counter_name",
)

nodes_raw.head()

### Cluster endpoints into nodes (snap grid)

In [ ]:
clustering = cluster_nodes_snap_grid(
    nodes_raw,
    tol_m=2,
    counter_col="counter_name",
)

nodes_raw = clustering.nodes_raw
node_points = clustering.node_points
segment_node_map = clustering.segment_node_map

print("Nodes (raw endpoints):", len(nodes_raw))
print("Nodes (clustered):", len(node_points))
segment_node_map.head()

### Define crossings (nodes with degree $\geq$ 3)

In [ ]:
crossing_ids = select_crossings_by_degree(
    nodes_raw,
    min_degree=3,
    counter_col="counter_name",
)

crossings_gdf = node_points[node_points["node_id"].isin(crossing_ids)].copy()

print("Crossings (degree >= 3):", len(crossings_gdf))
crossings_gdf.head()

### Assign accidents to nearest crossing

In [ ]:
acc_node, acc_node_ym = assign_accidents_to_nearest_crossing(
    joined_nearest_unique,
    crossings_gdf,
    max_distance_m=20,
)

print("Accidents assigned to crossings:", len(acc_node))
print("Accident groups (node×year×month):", len(acc_node_ym))
acc_node_ym.head()

### Build node-level exposure from segment flows

In [ ]:
node_exposure_ym = build_node_exposure_panel_from_segments(
    final_exposure_ym,
    segment_node_map,
    crossing_ids,
    trip_col="sum_strava_total_trip_count",
)

print("Node exposure (node×year×month) shape:", node_exposure_ym.shape)
node_exposure_ym.head()

### Combine into node-level risk panel

In [ ]:
node_panel_ym = build_node_risk_panel(
    node_exposure_ym,
    acc_node_ym,
    crossings_gdf,
)

print("Node panel (crossing x year x month) shape:", node_panel_ym.shape)
node_panel_ym.head()

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_node_panel.parquet"

node_panel_ym.to_parquet(
    out_path,
    index=False,
)

In [ ]:
# Look at the overall description of the dataset
node_panel_ym.describe(include="all")